# ΕΡΓΑΣΤΗΡΙΑΚΗ ΑΣΚΗΣΗ 1

Κάνουμε load τα αρχεία που χρειαζόμαστε για να τρέξουμε ζητούμενα βήματα.

Η συνάρτηση statistics υπολογίζει mean και variance για κάθε κλάση.

Η συνάρτηση accuracy είναι για να βγάζουμε ποσοστό επιτυχίας προβλέψεων.

In [1]:
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)
warnings.filterwarnings("ignore",category=FutureWarning)
path = '/home/zerkes/Desktop/DSML - MASTER/1st semester/PATTERN RECOGNITION/1 lab ex/pr_lab1_2016-17_data/'

import numpy as np

train = np.loadtxt(path + 'train.txt')
test =  np.loadtxt(path + 'test.txt')

x_train = train[:, 1::]
y_train = train[:, 0]
x_test = test[:, 1::]
y_test  = test[:, 0]

def statistics(X,y):               #calculates mean and variance for all digits
    variance = np.zeros((10,256))
    mean = np.zeros((10,256))
    for i in range(10):
        condition = [y == i]
        digit = X[condition]
        mean[i] = digit.mean(axis=0)
        variance[i]=digit.var(axis=0)
    return mean,variance
mean , variance = statistics(x_train,y_train)

def accuracy(X, y):

    acc_arr = (X == y)
    return (np.sum(acc_arr) / X.shape[0]) * 100

# ΒΗΜΑ 14

## Υπολογίστε τις a-priori πιθανότητες για κάθε κατηγορία.

### θα υπολογίσουμε τις a-priori πιθανότητες πάνω στο train set μόνο.

Υπολογίζουμε τις a-priori πιθανότητες με τον εξής τρόπο. Μετράμε πόσα δείγματα ανήκουν στη κάθε τάξη και διαιρούμε με το συνολικό πλήθος δειγμάτων. Ουσιαστικά οι apriori πιθανότητες είναι σα μια συνάρτηση συχνότητας της κλάσης.

In [2]:
a_priori = []
data_len = train.shape[0]
digit_len = []

for i in range(10):
    condition = [y_train == i] 
    digit = x_train[condition] # διαλέγω τη κλάση με το ψηφίο i
    digit_len.append(digit.shape[0]) # μετράω το πλήθος των δειγμάτων της.
digit_len

a_priori = [x / data_len for x in digit_len]  # τελικός υπολογισμός διαιρώ με το συνολικό πλήθος δειγμάτων
print(a_priori)

[0.16376354409546015, 0.13784117405019888, 0.10026059525442327, 0.09024825126868742, 0.08942531888629818, 0.07625840076807022, 0.09107118365107666, 0.08846523110684405, 0.07433822520916199, 0.08832807570977919]


In [3]:
sum(a_priori)

1.0

# ΒΗΜΑ 15

### Ταξινομήστε όλα τα ψηφία των test δεδομένων ως προς τις 10 κατηγορίες χρησιμοποιώντας τις τιμές της μέσης τιμής και διασποράς που υπολογίσατε στο Βήμα 9(α), υλοποιώντας έναν Bayesian ταξινομητή

In [4]:
from sklearn.base import BaseEstimator, ClassifierMixin
import numpy as np
import math
class BayesianClassifier(BaseEstimator, ClassifierMixin):  
    """Classify samples based on the distance from the mean feature value"""

    def __init__(self):
        self.X_mean_ = None
        self.X_var_ = None
        self.a_priori_ = None
        self.classes_ = None

    def fit(self, X, y):
        
        # find a-priori probabilities
        n_samples, n_features = X.shape
        self.classes_ = np.unique(y)
        n_classes = len(self.classes_)

        # calculate mean, var, and prior for each class
        self.X_mean_ = np.zeros((n_classes, n_features), dtype=np.float64)
        self.X_var_ = np.zeros((n_classes, n_features), dtype=np.float64)
        self.a_priori_ =  np.zeros(n_classes, dtype=np.float64)

        for c in range(len(self.classes_)):
            X_c = X[y==c]
            self.X_mean_[c] = X_c.mean(axis=0)
            self.X_var_[c] = X_c.var(axis=0)
            
            self.a_priori_[c] = X_c.shape[0] / float(n_samples)
        self.X_var_[self.X_var_ == 0] = 0.0001 # γιατί μετά θα κάνουμε διαίρεση με το 0 που δεν ορίζεται
         

    
    def pdf(self,class_idx,x):
        mean = self.X_mean_[class_idx]
        var = self.X_var_[class_idx]
        numerator = np.exp(-(x-mean)**2 / (2*var))
        denominator = np.sqrt(2* np.pi * var)
        return numerator / denominator
            
    
    
    def predict_function(self, X):
        """
        get the highest gaussianNB probability and classify the value in that class.
        """
        predictions= []
        posteriors = []
        for i in range(len(self.a_priori_)): # τρέχει για  i από 0-9
            prior = self.a_priori_[i]
            loginput = self.pdf(i,X)
            loginput[loginput == 0] = 0.0001 # γιατί το log(0) δεν ορίζεται
            class_conditional = np.sum(np.log(loginput))
            posterior = prior + class_conditional
            posteriors.append(posterior)
        return np.argmax(posteriors)
        
    def predict(self,X):
        y_pred = [self.predict_function(x) for x in X] # προέβλεψε για κάθε sample την κλάση του και βάλε την
        return np.array(y_pred)                          # στη λίστα.
            
      

    
    def scorer(self, X, y):
        """
        Return accuracy score on the predictions
        for X based on ground truth y
        """
        acc_arr = (X == y)
        return (np.sum(acc_arr) / X.shape[0]) * 100



### init:
        Στην init συνάρτηση ορίζω τις βασικές μεταβλητές μου που διαχειρίζομαι μέσα στην τάξη με το object που  δημιουργώ.
        
### scorer:
        Η συνάρτηση αυτή είναι ίδια με αυτή της προπαρασκευαστικής και απλώς υπολογίζει το accuracy των predictions. Δέχεται σα παραμέτρους τα predictions και τα actual values και αθρόιζει το πόσες προβλέψεις έπεσαν μέσα ενώ μετα διαιρούμε με τον μέγεθος του vector των prediction.
        
### fit:
        Στη συνάρτηση fit , περνάω τα x_train, y_train δεδομένα για να αρχίσει η διαδικασία εκπαιδεύσεως. Εκεί υπολογίζουμε το mean , variance για κάθε κλάση , επίσης της apriori πιθανότητες. Για μηδενικές τιμές του variance κάνω ενα replace με τιμές πολύ κοντά στο 0 , όπως 0.0001.
        
### predict:
         Η predict για κάθε vector του πίνακα X καλεί την predict_function , η οποία προβλέπει τν κλάση και την επιστρέφει σε μια λίστα με όλες τις προβλέψεις. Ουσιαστικά, όλη η δουλεία γίνεται στην predict_function.
 
### pdf:
        Η συνάρτηση pdf απλώς υπολογίζει τη γκαουσιανή πυκνότητα πιθανότητας, με τον εξής τρόπο. Αρχικά παίρνω τον πίνακα mean, variance για την  κλάση που περνάω σαν όρισμα. Τα διανύσματα αυτά ειναι vectors μεγέθους 256 όσα τα features. Έπειτα κάνουμε τις αντίστοιχες πράξεις που πρέπει με τον vector Χ που περάσαμε και τα mean , variance vectors. Φτιάχνουμε ξεχωριστά τον αριθμητή και τον παρανομαστή και μετά στο τέλος επιστρέφουμε τη διαίρεση τους.
        
### predict_function:
        Στη συνάρτηση αυτή υπολογίζω ουσιαστικά της posterior probabilities για την κάθε κλάση και επιλέγω την κλάση που έχει τη μεγαλύτερη τέτοια πιθανότητα. Για ευκολία, έχουμε χρησιμοποιήσει το λογαριθμικό τύπο. Ορίζουμε στην αρχή 2 λίστες μια για τα predictions και μια για τα posteriors. Στη 1η περνάμε την τελική πρόγνωση για ενα X vector ενώ στη 2η τις 10 posterior πιθανότητες για κάθε κλάση , ώστε να διαλέξουμε την μεγαλύτερη.
        Έπειτα, μέσα σε μια λούπα που τρέχει μια φορά για κάθε κλάση υπολογίζω τη pdf για την αντίστοιχη κλάση. Προβλέπουμε μετά την περίπτωση να βγει η πιθανότητα 0 , οπότε το αντικαθιστούμε με κάτι πολύ πολύ μικρό, 0.0001.
        Στη συνέχεια, αθροίζουμε τους λογαρίθμους των p(x1|y),....,p(x256|y) δηλαδή έχουμε , 
                  log(p(x1|y)) + .... + log(p(x256|y))
        Tέλος προσθέτουμε και την apriori πιθανότητα
                    log(p(x1|y)) + .... + log(p(x256|y)) + log(p(y))
        
        Η συνάρτηση επιστρέφει το index που έχει τη μεγαλύτερη τιμή στη λίστα posterior το οποίο το σώνει στον πίνακα predictions. Η συνάρτηση αυτή προβλέπει την κλάση για 1 vector κάθε φορά . Oυσιαστικά η predict την καλεί ανάλογα με τον αριθμό των vector που έχει να προβλέψει.

In [5]:
model1 = BayesianClassifier()

In [6]:
model1.fit(x_train,y_train)

In [7]:
pred = model1.predict(x_test)

In [9]:
model1.scorer(pred,y_test)

75.13702042850025

In [14]:
x_t

(7291,)

### Σύγκριση με τον naive bayes του scikit!

In [4]:
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
clf.fit(x_train, y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [23]:
pred1 = clf.predict(x_test)
acc_arr = (pred1 == y_test)
print((np.sum(acc_arr) / x_test.shape[0])*100) 

71.94818136522171


### Παρατηρούμε ότι η δική μας υλοποίηση είναι  κατά 3% καλύτερη!

# Βήμα 16
### Επαναλάβατε το Βήμα 15(α), (β) υποθέτοντας ότι η διασπορά για όλα τα χαρακτηριστικά, για όλες τις κατηγορίες ισούται με 1.

In [163]:
class BayesianClassifier2(BaseEstimator, ClassifierMixin):  
    """Classify samples based on the distance from the mean feature value"""

    def __init__(self):
        self.X_mean_ = None
        self.X_var_ = None
        self.a_priori_ = None
        self.classes_ = None

    def fit(self, X, y):
        
        # find a-priori probabilities
        n_samples, n_features = X.shape
        self.classes_ = np.unique(y)
        n_classes = len(self.classes_)

        # calculate mean, var, and prior for each class
        self.X_mean_ = np.zeros((n_classes, n_features), dtype=np.float64)
        self.X_var_ = np.ones((n_classes, n_features)) # θέτω με 1 όλο τον πίνακα var
        self.a_priori_ =  np.zeros(n_classes, dtype=np.float64)

        for c in range(len(self.classes_)):
            X_c = X[y==c]
            self.X_mean_[c] = X_c.mean(axis=0)
            
            
            self.a_priori_[c] = X_c.shape[0] / float(n_samples)
        #self.X_var_[self.X_var_ == 0] = 0.0001
         

    
    def pdf(self,class_idx,x):
        mean = self.X_mean_[class_idx]
        var = self.X_var_[class_idx]
        numerator = np.exp(-(x-mean)**2 / (2*var))
        denominator = np.sqrt(2* np.pi * var)
        return numerator / denominator
            
    
    
    def predict_function(self, X):
        """
        get the highest gaussianNB probability and classify the value in that class.
        """
        predictions= []
        posteriors = []
        for i in range(len(self.a_priori_)):
            prior = self.a_priori_[i]
            loginput = self.pdf(i,X)
            loginput[loginput == 0] = 0.0001
            class_conditional = np.sum(np.log(loginput))
            posterior = prior + class_conditional
            posteriors.append(posterior)
        return np.argmax(posteriors)
        
    def predict(self,X):
        y_pred = [self.predict_function(x) for x in X]
        return np.array(y_pred)
            
      

    
    def scorer(self, X, y):
        """
        Return accuracy score on the predictions
        for X based on ground truth y
        """
        acc_arr = (X == y)
        return (np.sum(acc_arr) / X.shape[0]) * 100



### H μόνη αλλαγή που έγινε στον classifier είναι στη μέθοδο fit να ορίζω τον πίνακα var ώς εξής:
    self.X_var_ = np.ones((n_classes, n_features)) # θέτω με 1 όλο τον πίνακα var

In [164]:
model2 = BayesianClassifier2()

In [165]:
model2.fit(x_train,y_train)

In [166]:
pred2 = model2.predict(x_test)

In [167]:
model2.scorer(pred2,y_test)

81.36522172396612

Παρατηρούμε πως το accuracy ανέβηκε κατα 5%. Πλέον η διασπορά βγαίνει από την εξίσωση καθώς δεν την επηρεάζει και το βάρος πέφτει πάνω στις μέσες τιμές για την πρόβλεψη.Πλέον για όλες τις κλάσεις οι τιμές που παίρνουν διασπείροντε γύρω από το μέσο τους ίσα. Το ποσοστό ανεβαίνει όπως φαίνεται και μας πάει κοντά σε αυτο του ΕuclideanClassifier().

Αυτό είναι λογικό καθώς αν πάρουμε για παράδειγμα δύο κλάσεις W1,W2 και θεωρήσουμε ότι έχουν ίδια διασπορά και αυτή είναι 1. Τότε αν εξισώσουμε της δύο εκ των υστέρων πιθανότητες θα καταλήξουμε στο ότι είναι ίσες αν  και μόνο αν , οι αποστάσεις τους από το μέσο είναι ίσες.

Επομένως με ίση διασπορά και στις 10 κλάσεις έχουμε την λειτουργία του Euclidean Classifier.


### παρατηρούμε πως με τις διακυμάνσεις όλες ίσες με 1 ανεβαίνει κατα το accuracy!

In [46]:
pred[pred2==pred].shape #

(1535,)

Με την παραπάνω εντολή βλέπουμε πόσα δείγματα προέβλεψαν και οι 2 ταξινομητές ίδια.
Παρακάτω φαίνεται το τι προέβλεψε ο 1ος ταξινομητής , τι ο 2ος με διασπορά 1 και τι είναι η πραγματική τιμή.

In [48]:
#for i,j in zip(pred,pred2):
#    print(i,"->" , j , "real: " , y_test[i])

9 -> 9 real:  9.0
6 -> 2 real:  0.0
3 -> 3 real:  6.0
0 -> 2 real:  9.0
6 -> 6 real:  0.0
0 -> 0 real:  9.0
0 -> 0 real:  9.0
0 -> 0 real:  9.0
6 -> 6 real:  0.0
9 -> 9 real:  9.0
6 -> 6 real:  0.0
2 -> 2 real:  3.0
3 -> 3 real:  6.0
4 -> 4 real:  6.0
4 -> 0 real:  6.0
8 -> 8 real:  6.0
1 -> 1 real:  6.0
2 -> 6 real:  3.0
9 -> 9 real:  9.0
6 -> 6 real:  0.0
2 -> 2 real:  3.0
2 -> 2 real:  3.0
9 -> 9 real:  9.0
9 -> 4 real:  9.0
6 -> 6 real:  0.0
2 -> 2 real:  3.0
0 -> 0 real:  9.0
3 -> 3 real:  6.0
8 -> 5 real:  6.0
3 -> 3 real:  6.0
7 -> 7 real:  0.0
0 -> 0 real:  9.0
5 -> 5 real:  0.0
0 -> 0 real:  9.0
7 -> 7 real:  0.0
9 -> 9 real:  9.0
8 -> 4 real:  6.0
2 -> 0 real:  3.0
0 -> 0 real:  9.0
7 -> 7 real:  0.0
0 -> 0 real:  9.0
2 -> 2 real:  3.0
1 -> 1 real:  6.0
0 -> 0 real:  9.0
7 -> 7 real:  0.0
1 -> 1 real:  6.0
0 -> 0 real:  9.0
9 -> 9 real:  9.0
2 -> 2 real:  3.0
0 -> 0 real:  9.0
8 -> 3 real:  6.0
5 -> 5 real:  0.0
6 -> 1 real:  0.0
2 -> 0 real:  3.0
0 -> 0 real:  9.0
6 -> 6 rea

# ΒΗΜΑ 17

### Σύγκριση με svm και knn

# SVM:


### kernel = rbf

In [88]:
from sklearn.svm import SVC
clf = SVC(kernel='rbf')
clf.fit(x_train,y_train)
pred_svm = clf.predict(x_test)
print(accuracy(pred_svm,y_test))

94.22022919780767


### kernel = linear

In [87]:
from sklearn.svm import SVC
clf = SVC(kernel='linear')
clf.fit(x_train,y_train)
pred_svm = clf.predict(x_test)
print(accuracy(pred_svm,y_test))

92.62580966616841


### kernel = poly

In [93]:
from sklearn.svm import SVC
clf = SVC(kernel='poly')
clf.fit(x_train,y_train)
pred_svm = clf.predict(x_test)
print(accuracy(pred_svm,y_test))

94.66865969108122


Παρατηρούμε πως το svm για kernel poly ή rbf έχει καλύτερα αποτελέσματα από ότι ο linear. Πράγμα που είναι λογικό καθώς τα δεδομένα μας δεν είναι γραμμικώς διαχωρίσιμα οπότε τα non-linear kernels αποδίδουν καλύτερα.
Το μειονέκτημα όμως τον non-linear kernel είναι η υπολογιστική πολυπλοκότητα.

# K-NN

Παρακάτω δοκιμάζουμε τον αλγόριθμο knn με τη χρήση ενός NCA distance metric learning algorithm για να βελτιωθεί η χρήση του αλγορίθμου knn.  O αλγόριθμος nca μας βοηθάει σε περιπτώσεις multi class classification χωρίς να μεγαλώνει το μοντέλο μας με επιπλέον παραμέτρους.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier , NeighborhoodComponentsAnalysis

from sklearn.pipeline import Pipeline
knn = KNeighborsClassifier(n_neighbors=3)
nca = NeighborhoodComponentsAnalysis(random_state=42)
trans_pipe = Pipeline([('nca',nca),('knn',knn)])
trans_pipe.fit(x_train,y_train)
pred_nca_knn = trans_pipe.predict(x_test)

In [92]:
print(accuracy(pred_nca_knn,y_test))

95.21674140508222


# ΒΗΜΑ 18

### (a)

    Οι αλγόριθμοι που συνδυάζουμε είναι  ο knn με χρήση pca και nca το οποίο το έχουμε χτίσει σε ενα pipeline.
    Έπιπλέον έχουμε τον αλγόριθμο svm με kernel = 'poly' και ένα randomforestclassifier.
    
    Έπιλέγουμε την παράμετρο voting = soft πράγμα που σημαίνει ότι πρέπει να περάσουμε ένα διάνυσμα βαρών για  τις πιθανότητες της κάθε κλάσης. Δίνουμε έμφαση κύριως στον KNN που είχε τα καλύτερα αποτελέσματα δίνοντας του βάρος = 2.

In [7]:
from sklearn.ensemble import VotingClassifier , RandomForestClassifier
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA



In [124]:
#clf1= KNeighborsClassifier(n_neighbors=3)
pca = PCA(n_components=50)
clf1 = Pipeline([('pca',pca),('nca',nca),('knn',knn)])
clf2= svm.SVC(kernel='poly' , probability=True )
clf3= RandomForestClassifier(n_estimators=50, random_state=1)


eclf1 = VotingClassifier(estimators=[('knn', clf1), ('svm.SVC', clf2), ('rf', clf3)], voting='soft',weights=[2,1,1.3])


In [125]:
eclf1.fit(x_train,y_train)

VotingClassifier(estimators=[('knn',
                              Pipeline(memory=None,
                                       steps=[('pca',
                                               PCA(copy=True,
                                                   iterated_power='auto',
                                                   n_components=50,
                                                   random_state=None,
                                                   svd_solver='auto', tol=0.0,
                                                   whiten=False)),
                                              ('nca',
                                               NeighborhoodComponentsAnalysis(callback=None,
                                                                              init='auto',
                                                                              max_iter=50,
                                                                              n_components=None,
           

In [126]:
print(accuracy(eclf1.predict(x_test),y_test))

95.31639262580967


Παρατηρούμε πως το accuracy αυξήθηκε στο 95.31 από 95.2 που ήταν  συνδυασμός nca και knn classifier.

Ο λόγος που πρέπει να χρησιμοποιούμε μονό αριθμό ταξινομητών είναι για να μην υπάρχει ισοβαθμία σε περίπτωση που ο ένας ψηφίζει τη μια τάξη και ο άλλος την άλλη. Έτσι με μονό αριθμό ταξινομητών έχουμε καλύτερα αποτελέσματα. 

### (b)

## Bagging Classifier

    Αρχικά θα τρέξουμε τον knn μόνο του να δούμε τι accuracy θα μας δώσει. Στη συνέχεια θα το τρέξουμε αφού τον έχουμε εκπαιδεύσει με το Bagging Classifier.

In [8]:
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(x_train,y_train)
knnpred = knn.predict(x_test)
print(accuracy(knnpred,y_test))

94.46935724962631


In [9]:
from sklearn.ensemble import BaggingClassifier
bagging = BaggingClassifier(KNeighborsClassifier(),max_samples=0.5, max_features=1.0 ,n_estimators=3)

In [10]:
bagging.fit(x_train,y_train)

BaggingClassifier(base_estimator=KNeighborsClassifier(algorithm='auto',
                                                      leaf_size=30,
                                                      metric='minkowski',
                                                      metric_params=None,
                                                      n_jobs=None,
                                                      n_neighbors=5, p=2,
                                                      weights='uniform'),
                  bootstrap=True, bootstrap_features=False, max_features=1.0,
                  max_samples=0.5, n_estimators=3, n_jobs=None, oob_score=False,
                  random_state=None, verbose=0, warm_start=False)

In [12]:
predbag = bagging.predict(x_test)

In [13]:
accuracy(predbag,y_test)

93.17389138016941

Παρατηρούμε πως το accuracy μειώθηκε για πολύ λίγο σε ποσοστό 1% περίπου.

### Τώρα θα δοκιμάσουμε το παράδειγμα της εργασίας για τα decision tree και τα random forest

Τρέχουμε αρχικά το implementation της scikit για το random forest ώστε να δούμε τι ποσοστό δίνει.

In [3]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=10)
clf = clf.fit(x_train,y_train)
predforest = clf.predict(x_test)
accuracy(predforest,y_test)

90.3836571998007

Τώρα τρέχουμε το decision tree

In [145]:
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier()
dtc.fit(x_train,y_train)
preddtc = dtc.predict(x_test)
accuracy(preddtc,y_test)

82.31190832087692

Tέλος , τρέχουμε το bagging classifier με παράμετρο to decision tree και για κάθε bag διαλέγει το 50 τις εκατό των δειγμάτων με επανατοποθέτηση(bootstrap) ενώ παίρνει και το 70 τις εκατό των χαρακτηριστικών!

Η default παράμετρος n_estimators που λέει πόσους base estimators να βάλουμε είναι 10. Επομένως, είναι σαν να παίρνει 10 decision trees και να το κάνει το κάθε ένα train σε ένα sample που περιέχει το 50% του training dataset με επανατοποθέτηση.

Επομένως είναι σα να δημιουργούμε ενα random forest με 10 trees.

In [14]:
from sklearn.tree import DecisionTreeClassifier
bagging = BaggingClassifier(DecisionTreeClassifier(),max_samples=0.5, max_features=0.7) #bootstrap is true default
bagging.fit(x_train,y_train)
predbag = bagging.predict(x_test)
accuracy(predbag,y_test)

0.8898854010961634

## (c) ΣΥΜΠΕΡΑΣΜΑΤΑ

Παρατηρούμε πως στην περίπτωση του decision tree , η χρήση του baggingClassifier έφτασε πραγματικά πολύ κοντά στην υλοποιήση για τα random forest και με χρήση λιγότερων features(70% του συνόλου.).

Στην περίπτωση του knn δεν είδαμε διαφορά , ίσως γιατί ο αλγόριθμος ήδη απέδιδε πάρα πολύ καλά.

Η τεχνική του baggingClassifier με τη χρήση bootstrap sampling μας βοηθάει στο να γενικοποιήσουμε καλύτερα το μοντέλο μας με μια τυχαιότητα.

Ο τρόπος που λειτουργεί δηλαδή είναι να πάρει πολλά δείγματα ενός ποσοστού του συνολικού και να τα χωρίσει σε κάποια υποσύνολα. Για κάθε υποσύνολο κάνει  training το μοντέλο και έπειτα συμψηφίζονται τα τελικά χαρακτηριστικά του μοντέλου.



# BHMA 19 

### (α)
Υλοποίηση ενός dataloader για τα δεδομένα μας.
Αρχικά έχουμε έναν dataloader για τα training δεδομένα.
Έπειτα έχουμε εναν dataloader για τα test δεδομένα.
Γίνεται χρήση batch_size = 1 ώστε να γίνονται update οι παράμετροι του νευρωνικού  1 φορά κάθε φορά που κάνει training όλα τα feature vectors. Ουσιαστικά για να τρέξουμε τον αλγόριθμο του mini_batch θέλει batch_size>1. Τρέχοντας το νευρωνικό με batch_size=1 αργεί παραπάνω σε χρόνο αλλά κάνει καλύτερο training. Ώστοσο λόγω του όγκου των δεδομένων δοκιμάζουμε να το τρέξουμε με όλο το dataset καθώς δε μας επηρεάζει ιδιαίτερα.

In [31]:
import torch
import torch.utils.data as utils
tensor_x_train = torch.stack([torch.Tensor(i) for i in x_train])
tensor_y_train = torch.stack([torch.Tensor(np.array(i)) for i in y_train])

tensor_x_test = torch.stack([torch.Tensor(i) for i in x_test])
tensor_y_test = torch.stack([torch.Tensor(np.array(i)) for i in y_test])

trainset = utils.TensorDataset(tensor_x_train,tensor_y_train)
trainloader =  utils.DataLoader(trainset, batch_size = 1, shuffle=True)

testset  = utils.TensorDataset(tensor_x_test,tensor_y_test)
testloader = utils.DataLoader(testset, batch_size=1,shuffle=False)

### (b)
Παρακάτω φαίνεται η υλοποίηση ενός fully connected νευρωνικού δικτύου.
Η αριτεκτονική του έχει ως εξής:
    input layer -> 256 ,64
    2 hidden layer -> 64,64
    1 output layer -> 64,10 
 Ο λόγος που έχουμε 256 στο input είναι λόγω των features και ο λόγος που έχουμε 10 στο output είναι ο αριθμός των κλάσεων που προβλέπουμε.
 
Τα πρώτα 3 layers  κάνουν linear transformation και έχουν activation function την relu η οποία έχει την εξής λειτουργία:
    Relu(x) = max(0,x)
To τελευταίο layer καλέι την softmax για να μετατραπούν τα outputs σε αυτό το layer σε πιθανότητες και να επιλέξουμε τη μέγιστη, ώστε να  διαλέξουμε την αντίστοιχη κλάση.

In [5]:
import torch.nn as nn
import torch.nn.functional as F

In [40]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(16*16, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 64)
        self.fc4 = nn.Linear(64, 10)
    def forward(self,x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        
        return F.log_softmax(x , dim=1)
        

In [41]:
net = Net()

Παρακάτω βλέπουμε την εκπαίδευση του νευρωνικού δικτύου. Γίνεται χρήση του optimizer adam με learning rate 0.001. 

Το training θα γίνει 10 φορες σε όλα τα δεδομένα και ανά 1 batch γίνεται update στις παραμέτρους του νευρωνικού δικτύου γιατί δηλώσαμε batch_size = 10 στο dataloader στην αρχή.

Επίσης γίνεται χρήση της cross entropy loss function καθώς κρίνεται κατάλληλη για outputs πιθανοτήτων από softmax.

In [42]:
import torch.optim as optim
optimizer = optim.Adam(net.parameters(), lr=0.0001)

EPOCHS = 5

for epoch in range(EPOCHS):
    for data in trainloader: # data is a batch of featuresets and labels
        X,y = data
        net.zero_grad() # making grad zero so it does not sum up.
        output = net(X.view(-1,16*16))
        loss = F.nll_loss(output,torch.tensor(y, dtype=torch.long))
        loss.backward()
        optimizer.step()
    print(loss)
        
        


/home/zerkes/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # This is added back by InteractiveShellApp.init_path()


tensor(0.1132, grad_fn=<NllLossBackward>)
tensor(0.0043, grad_fn=<NllLossBackward>)
tensor(0.0024, grad_fn=<NllLossBackward>)
tensor(0.0002, grad_fn=<NllLossBackward>)
tensor(0.1134, grad_fn=<NllLossBackward>)


### (d)
Εδώ βλέπουμε την απόδοση του νευρωνικού μας δικτύου στα test δεδομένα.

In [43]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        X,y = data
        output = net(X.view(-1,16*16))
        for idx, i in enumerate(output):
            if torch.argmax(i) == y[idx]:
                correct +=1 
            total +=1
    
print("Accuracy score is: " , round(correct/total,5)*100)

Accuracy score is:  91.878


### (c) ΥΛΟΠΟΙΗΣΗ ΣΕ SCIKIT και evaluation στο test set

Στον παρακάτω κώδικα, έχουμε στην init function τις αρχικοποιήσεις με none του self.net, self.loss,self.optimizer οι οποίες είναι οι κύριες μεταβλητές του μοντέλου μας.

Στη συνάρτηση fit , γίνεται το training του νευρωνικού μας δικτύου,ενώ ταυτόχρονα μετατρέπουμε τα Χ,Υ δεδομένα που δέχεται σε ενα dataloader, Το οποίο έχει batch_size=10(Διαφορετικό απο το προηγούμενο ερώτημα που βάλαμε 1). Επίσης κάνουμε ενα shuffle τα δεδομένα και έπειτα ξεκινάει η εκπαίδευση για 10 epochs δηλαδή 10 περάσματα του συνόλου δεδομένων που έχουμε.

Η συνάρτηση scorer είναι ίδια.

Η συνάρτηση predict , ουσιαστικά ειναι το forward κομμάτι του νευρωνικού δικτύου. Μετατρέπουμε το x_test από numpy σε torch.tensor ώστε να είναι συμβατό με το pytorch. Έπειτα παίρνουμε κάθε row της ξεχωριστά , που είναι ενα feature vector και βρίσκουμε το output του μετά το forward function. 
    Tέλος επιλέγουμε το index με την υψηλότερη πιθανότητα από τις 10 που επιστρέφει, μια για την κάθε κλάση.
    

In [44]:
import torch.nn as nn
import torch.optim as optim
import torch
import torch.utils.data as utils
import torch.nn.functional as F
from sklearn.base import BaseEstimator, ClassifierMixin
import numpy as np
class NNClassifier(BaseEstimator, ClassifierMixin):  
    """Classify samples based on the distance from the mean feature value"""

    def __init__(self):
        self.net = None
        self.loss= None
        self.optimizer = None 


    def fit(self, X, y):
        self.net = Net()
        self.optimizer = optim.Adam(self.net.parameters(), lr=0.0001)
        tensor_x_train = torch.stack([torch.Tensor(i) for i in X])
        tensor_y_train = torch.stack([torch.Tensor(np.array(i)) for i in y])

        trainset = utils.TensorDataset(tensor_x_train,tensor_y_train)
        trainloader =  utils.DataLoader(trainset, batch_size = 10, shuffle=True)

        EPOCHS = 10

        for epoch in range(EPOCHS):
            for data in trainloader: # data is a batch of featuresets and labels
                X,y = data
                self.net.zero_grad() # making grad zero so it does not sum up.
                output = self.net(X.view(-1,16*16))
                self.loss = F.nll_loss(output,torch.tensor(y, dtype=torch.long))
                self.loss.backward()
                self.optimizer.step()
            print(self.loss)
        
        
        return self


    def predict(self, X):
        tensor_x_test = torch.stack([torch.Tensor(i) for i in X])

        pred = []
        with torch.no_grad():
            for data in tensor_x_test:
                X  = data
                myoutput= self.net(X.view(-1,16*16))
                pred.append(torch.argmax(myoutput))
        return np.asarray(pred)        
                    
        
    
    def scorer(self, X, y):
        acc_arr = (X == y)
        return (np.sum(acc_arr) / X.shape[0]) * 100

In [11]:
nnclf = NNClassifier()
nnclf.fit(x_train,y_train)

/home/zerkes/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


tensor(0.4395, grad_fn=<NllLossBackward>)
tensor(0.0205, grad_fn=<NllLossBackward>)
tensor(2.7920, grad_fn=<NllLossBackward>)
tensor(0.0938, grad_fn=<NllLossBackward>)
tensor(0.2972, grad_fn=<NllLossBackward>)
tensor(0.0064, grad_fn=<NllLossBackward>)
tensor(0.7744, grad_fn=<NllLossBackward>)
tensor(0.0016, grad_fn=<NllLossBackward>)
tensor(0.0840, grad_fn=<NllLossBackward>)
tensor(0.0119, grad_fn=<NllLossBackward>)


NNClassifier()

In [12]:
nnpred = nnclf.predict(x_test)

In [13]:
print(nnclf.scorer(nnpred,y_test))


91.52964623816642


In [14]:
nnpred

array([9, 6, 3, ..., 4, 0, 1])